# Introduction:
Dates and times (datetimes) are frequently encountered during preprocessing for
machine learning, whether the time of a particular sale or the year of some public
health statistic. In this notebook, we will build a toolbox of strategies for handling time
series data including tackling time zones and creating lagged time features. Specifically,
we will focus on the time series tools in the pandas library, which centralizes the
functionality of many other libraries.

When we look at a date time stamp, a number of features, or pieces of information are immediately obvious:
###### * Year
###### * Month
###### * Day
###### * Day of week
###### * Week of year
###### * Hour of day

Month and day of week can be quite useful in understanding periodicity or seasonality of transactions. We may find that some actions are more probable on certain days of the week, or somethings happen around the same month every year. For example: During Diwali, everyone used to buy gold ornaments (Atleast in India).

###### Specifically, we will focus on the time series tools in the pandas library, which centralizes the functionality of many other libraries.

# -1- Converting string into date
## Problem:
Given vector of strings as dates and times, you want to transform them into time series data.

## Solution:
Use pandas' to_datetime with format of date. 

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Create strings
date_col = np.array(['03-04-2005 11:35 PM',
                        '23-05-2010 12:01 AM',
                        '04-09-2009 09:09 PM'])

In [3]:
# Convert to datetimes
date_col = [pd.to_datetime(date, format = '%d-%m-%Y %I:%M %p', errors = 'coerce') for date in date_col]
date_col

[Timestamp('2005-04-03 23:35:00'),
 Timestamp('2010-05-23 00:01:00'),
 Timestamp('2009-09-04 21:09:00')]

If errors="coerce", then any problem that occurs will not raise an error (the default
behavior) but instead will set the value causing the error to NaT (i.e., a missing value).

## Discussion:
One obstacle to strings representing dates and
times is that the format of the strings can vary significantly between data sources. For
example, one vector of dates might represent March 23rd, 2015 as “03-23-15” while
another might use “3|23|2015”. We can use the format parameter to specify the exact
format of the string.

Here are some common date and time formatting codes:
#### Code           -             Description     -        Example
#### %Y              -              Full year          -           2001
#### %m             -      Month w/ zero padding  -       04
#### %d    -   Day of the month w/ zero padding -  09
#### %I      -  Hour (12hr clock) w/ zero padding -  02
#### %p        -                   AM or PM             -           AM
#### %M        -           Minute w/ zero padding  -      05
#### %S          -         Second w/ zero padding  -     09



More detailed list available at: http://strftime.org/

# -2- Handling time zones

## Problem:
You have time series data and want to add or change time zone information.

## Solution:
If not specified, pandas objects have no time zone. However, we can add a time zone
using tz during creation:

In [4]:
import pandas as pd
pd.Timestamp('2017-05-01 06:00:00', tz='Europe/London')

Timestamp('2017-05-01 06:00:00+0100', tz='Europe/London')

In [5]:
# To see all available timezones:
import pytz
pytz.all_timezones[:]

['Africa/Abidjan',
 'Africa/Accra',
 'Africa/Addis_Ababa',
 'Africa/Algiers',
 'Africa/Asmara',
 'Africa/Asmera',
 'Africa/Bamako',
 'Africa/Bangui',
 'Africa/Banjul',
 'Africa/Bissau',
 'Africa/Blantyre',
 'Africa/Brazzaville',
 'Africa/Bujumbura',
 'Africa/Cairo',
 'Africa/Casablanca',
 'Africa/Ceuta',
 'Africa/Conakry',
 'Africa/Dakar',
 'Africa/Dar_es_Salaam',
 'Africa/Djibouti',
 'Africa/Douala',
 'Africa/El_Aaiun',
 'Africa/Freetown',
 'Africa/Gaborone',
 'Africa/Harare',
 'Africa/Johannesburg',
 'Africa/Juba',
 'Africa/Kampala',
 'Africa/Khartoum',
 'Africa/Kigali',
 'Africa/Kinshasa',
 'Africa/Lagos',
 'Africa/Libreville',
 'Africa/Lome',
 'Africa/Luanda',
 'Africa/Lubumbashi',
 'Africa/Lusaka',
 'Africa/Malabo',
 'Africa/Maputo',
 'Africa/Maseru',
 'Africa/Mbabane',
 'Africa/Mogadishu',
 'Africa/Monrovia',
 'Africa/Nairobi',
 'Africa/Ndjamena',
 'Africa/Niamey',
 'Africa/Nouakchott',
 'Africa/Ouagadougou',
 'Africa/Porto-Novo',
 'Africa/Sao_Tome',
 'Africa/Timbuktu',
 'Africa/

In [6]:
# We can add a time zone to a previously created datetime using tz_localize:
date = pd.Timestamp('2017-05-01 06:00:00')
date_in_Abidjan = date.tz_localize('Africa/Abidjan')
date_in_Abidjan

Timestamp('2017-05-01 06:00:00+0000', tz='Africa/Abidjan')

In [7]:
# We can change timezone of previously created timestamp
date_in_Abidjan.tz_convert('Europe/London')

Timestamp('2017-05-01 07:00:00+0100', tz='Europe/London')

In [8]:
# pandas’ Series objects can apply tz_localize and tz_convert to every element:

# Create three dates
dates = pd.Series(pd.date_range('2/2/2002', periods=3, freq='M'))

# Set time zone
dates.dt.tz_localize('Africa/Abidjan')

0   2002-02-28 00:00:00+00:00
1   2002-03-31 00:00:00+00:00
2   2002-04-30 00:00:00+00:00
dtype: datetime64[ns, Africa/Abidjan]

# -3- Selecting date and times
## Problem:
You have a vector of dates and you want to select one or more.

## Solution:
Use two boolean conditions as the start and end dates:

In [9]:
import pandas as pd

# Create empty dataframe
df = pd.DataFrame()

df['date'] = pd.date_range('1/1/2019', periods=100000, freq='H')
print(df.shape)
df.head()

(100000, 1)


,date
0,2019-01-01 00:00:00
1,2019-01-01 01:00:00
2,2019-01-01 02:00:00
3,2019-01-01 03:00:00
4,2019-01-01 04:00:00


In [10]:
# Select observations between two datetimes
df[(df['date'] > '2019-1-1 01:00:00') & (df['date'] <= '2019-1-1 10:00:00')]

,date
2,2019-01-01 02:00:00
3,2019-01-01 03:00:00
4,2019-01-01 04:00:00
5,2019-01-01 05:00:00
6,2019-01-01 06:00:00
7,2019-01-01 07:00:00
8,2019-01-01 08:00:00
9,2019-01-01 09:00:00
10,2019-01-01 10:00:00


Alternatively we can set date column as dataframe's index and then slice using loc

In [11]:
# Set index
df = df.set_index(df['date'])

# Select observations between two datetimes
df.loc['2019-1-1 01:00:00':'2019-1-1 04:00:00']

,date
date,
2019-01-01 01:00:00,2019-01-01 01:00:00
2019-01-01 02:00:00,2019-01-01 02:00:00
2019-01-01 03:00:00,2019-01-01 03:00:00
2019-01-01 04:00:00,2019-01-01 04:00:00


# -4- Breaking Up Date Data into Multiple Features

## Problem:
You have a column of dates and times and you want to create features for year,
month, day, hour, and minute.

## Solution:
Use pandas Series.dt’s time properties:


In [12]:
# Load library
import pandas as pd
# Create data frame
dataframe = pd.DataFrame()
# Create five dates
dataframe['date'] = pd.date_range('1/1/2001', periods=150, freq='W')

In [13]:
# Create features for year, month, day, hour, and minute
dataframe['year'] = dataframe['date'].dt.year
dataframe['month'] = dataframe['date'].dt.month
dataframe['day'] = dataframe['date'].dt.day
dataframe['hour'] = dataframe['date'].dt.hour
dataframe['minute'] = dataframe['date'].dt.minute

dataframe.head()

,date,year,month,day,hour,minute
0,2001-01-07,2001,1,7,0,0
1,2001-01-14,2001,1,14,0,0
2,2001-01-21,2001,1,21,0,0
3,2001-01-28,2001,1,28,0,0
4,2001-02-04,2001,2,4,0,0


Sometimes it can be useful to break up a column of dates into components. For
example, we might want a feature that just includes the year of the observation or we
might want only to consider the month of some observation so we can compare them
regardless of year.

# -5- Calculating the Difference Between Dates
## Problem:
You have two datetime features and want to calculate the time between them for each
observation.
## Solution:
Subtract the two date features using pandas:

In [14]:
# Load library
import pandas as pd
# Create data frame
dataframe = pd.DataFrame()
# Create two datetime features
dataframe['Arrived'] = [pd.Timestamp('01-01-2017'), pd.Timestamp('01-04-2017')]
dataframe['Left'] = [pd.Timestamp('01-01-2017'), pd.Timestamp('01-06-2017')]

dataframe.head()

,Arrived,Left
0,2017-01-01,2017-01-01
1,2017-01-04,2017-01-06


In [15]:
dataframe['Left'] - dataframe['Arrived']

0   0 days
1   2 days
dtype: timedelta64[ns]

In [16]:
dataframe['difference'] = [tmp.days for tmp in (dataframe['Left'] - dataframe['Arrived'])]
dataframe.head()

,Arrived,Left,difference
0,2017-01-01,2017-01-01,0
1,2017-01-04,2017-01-06,2


## Discussion:
There are times when the feature we want is the change (delta) between two points in
time. For example, we might have the dates a customer checks in and checks out of a
hotel, but the feature we want is the duration of his stay. pandas makes this calculation
easy using the TimeDelta data type.

# -6- Encoding Days of the Week
## Problem:
You have a vector of dates and want to know the day of the week for each date.

## Solution:
Use pandas’ Series.dt property weekday_name:

In [17]:
# Load library
import pandas as pd

# Create dates
dates = pd.Series(pd.date_range("2/2/2002", periods=3, freq="M"))

# Show days of the week
dates.dt.weekday_name

0    Thursday
1      Sunday
2     Tuesday
dtype: object

If we want the output to be a numerical value and therefore more usable as a machine
learning feature, we can use weekday where the days of the week are represented as an
integer (Monday is 0):

In [18]:
# Show days of the week
dates.dt.weekday

0    3
1    6
2    1
dtype: int64

# -7- Creating a Lagged Feature
## Problem:
You want to create a feature that is lagged n time periods.

## Solution:


In [19]:
# Load library
import pandas as pd

# Create data frame
dataframe = pd.DataFrame()

# Create data
dataframe["dates"] = pd.date_range("1/1/2001", periods=5, freq="D")
dataframe["stock_price"] = [1.1,2.2,3.3,4.4,5.5]

# Lagged values by one row
dataframe["previous_days_stock_price"] = dataframe["stock_price"].shift(1)

# Show data frame
dataframe

,dates,stock_price,previous_days_stock_price
0,2001-01-01,1.1,NaN
1,2001-01-02,2.2,1.1
2,2001-01-03,3.3,2.2
3,2001-01-04,4.4,3.3
4,2001-01-05,5.5,4.4


Very often data is based on regularly spaced time periods (e.g., every day, every hour,
every three hours) and we are interested in using values in the past to make predictions
(this is often called lagging a feature). For example, we might want to predict a
stock’s price using the price it was the day before. With pandas we can use shift to
lag values by one row, creating a new feature containing past values.
In our solution, the first row for previous_days_stock_price is a missing value
because there is no previous stock_price value.

# -8- Using Rolling Time Windows
## Problem:
Given time series data, you want to calculate some statistic for a rolling time.
## Solution:

In [20]:
# Load library
import pandas as pd

# Create datetimes
time_index = pd.date_range("01/01/2010", periods=5, freq="M")

# Create data frame, set index
dataframe = pd.DataFrame(index=time_index)

# Create feature
dataframe["Stock_Price"] = [1,2,3,4,5]

In [21]:
# Calculate rolling mean
dataframe.rolling(window=2).mean()

,Stock_Price
2010-01-31,NaN
2010-02-28,1.5
2010-03-31,2.5
2010-04-30,3.5
2010-05-31,4.5


Rolling (also called moving) time windows are conceptually simple but can be difficult
to understand at first. Imagine we have monthly observations for a stock’s price.
It is often useful to have a time window of a certain number of months and then
move over the observations calculating a statistic for all observations in the time window.
For example, if we have a time window of three months and we want a rolling mean,
we would calculate:

###### 1. mean(January, February, March)
###### 2. mean(February, March, April)
###### 3. mean(March, April, May)
###### Likewise

###### Another way to put it: our three-month time window “walks” over the observations, calculating the window’s mean at each step.

###### pandas’ rolling allows us to specify the size of the window using window and then quickly calculate some common statistics, including the max value (max()), mean value (mean()), count of values (count()), and rolling correlation (corr()). Rolling means are often used to smooth out time series data because using the mean of the entire time window dampens the effect of short-term fluctuations.

# -9- Handling Missing Data in Time Series
## Problem:
You have missing values in time series data.

## Solution:
In addition to the missing data strategies previously discussed, when we have time
series data we can use interpolation to fill in gaps caused by missing values:

In [22]:
# Load libraries
import pandas as pd
import numpy as np

# Create date
time_index = pd.date_range("01/01/2010", periods=5, freq="M")

# Create data frame, set index
dataframe = pd.DataFrame(index=time_index)

# Create feature with a gap of missing values
dataframe["Sales"] = [1.0,2.0,np.nan,np.nan,5.0]

# Interpolate missing values
dataframe.interpolate()

,Sales
2010-01-31,1.0
2010-02-28,2.0
2010-03-31,3.0
2010-04-30,4.0
2010-05-31,5.0


Alternatively, we can replace missing values with the last known value (i.e., forwardfilling):

In [23]:
dataframe.ffill()

,Sales
2010-01-31,1.0
2010-02-28,2.0
2010-03-31,2.0
2010-04-30,2.0
2010-05-31,5.0


We can also replace missing values with the latest known value (i.e., back-filling):

In [24]:
dataframe.bfill()

,Sales
2010-01-31,1.0
2010-02-28,2.0
2010-03-31,5.0
2010-04-30,5.0
2010-05-31,5.0


If we believe the line between the two known points is nonlinear, we can use interpo
late’s method to specify the interpolation method:

In [25]:
dataframe.interpolate(method="quadratic")

,Sales
2010-01-31,1.000000
2010-02-28,2.000000
2010-03-31,3.059808
2010-04-30,4.038069
2010-05-31,5.000000


That's it guys for today. Stay in contact for next notebook. Until then, enjoy data analytics... :)